In [9]:
import pandas as pd
import numpy as np
import scipy as sp
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
def pdf_fig(figure, filename):
    figure = figure.get_figure()
    figure.savefig("../../thesis/img/big-study/" + filename + ".pdf", bbox_inches = "tight")

In [11]:
# seaborn styling
sns.set_style("darkgrid")
sns.set_palette(sns.husl_palette(h=.5, s=1))
sns.set_context("talk")

def label_bars_ints(ax):
    for p in ax.patches:
        ax.annotate("%.0i" % np.nan_to_num(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()),
            ha='center', va='center', fontsize=14, color='black', xytext=(0, 10),
            textcoords='offset points')

def label_bars_floats(ax):
    for p in ax.patches:
        ax.annotate("%.2f" % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()),
            ha='center', va='center', fontsize=14, color='black', xytext=(0, 10),
            textcoords='offset points')

def process_fig(ax, name, technique, int_annotation, hue_title):
    if (int_annotation):
        label_bars_ints(ax)
    else:
        label_bars_floats(ax)
    if (hue_title != ''):
        ax.legend(loc='center left', bbox_to_anchor=(1.02, 0.5), title=hue_title)
    pdf_fig(ax, name + '-' + technique)
    plt.show()
    ax.clear()

In [12]:
techniques = ['PBE', 'TS', 'SKWS', 'RLR']
bool_order = [True, False]

In [13]:
pbe = pd.read_csv('pbe.csv')
pbe['Technique'] = 'PBE'
ir = pd.read_csv('ir.csv')
ir['Technique'] = 'TS'
kw = pd.read_csv('keyword.csv')
kw['Technique'] = 'SKWS'
rn = pd.read_csv('random.csv')
rn['Technique'] = 'RLR'
frames = [pbe, ir, kw, rn]

result_big = pd.concat(frames)
result_big.rename(columns={'Unnamed: 0':'v',
                       'TestOutput':'Test Output',
                       'DesiredTestOutput':'Desired Test Output',
                       'ExampleCount':'Example Count',
                       'LearningDuration':'Learning Duration',
                       'ApplicationDuration':'Application Duration'}, inplace=True)

def calc_overlapping_lines(row):
    lines_desired = str(row['Desired Test Output']).splitlines()
    lines_output = str(row['Test Output']).splitlines()
    overlap = list((Counter(lines_output) & Counter(lines_desired)).elements())
    row['True Positives Count'] = len(overlap)
    row['False Positives Count'] = len(lines_output) - len(overlap)
    row['False Negatives Count'] = len(lines_desired) - len(overlap)
    row['Output Lines Count'] = len(lines_output)
    row['Desired Lines Count'] = len(lines_desired)
    row['All Desired Lines Extracted'] = row['False Negatives Count'] == 0
    return row

result_big = result_big.apply(calc_overlapping_lines, axis=1)

result_big.head()

,v,Example Count,LearnedProgram,TestInputPath,Test Output,Desired Test Output,Accuracy,IoU,Successful,Learning Duration,...,TestInputLineCount,TestCategory,ContextSizeFactor,Technique,True Positives Count,False Positives Count,False Negatives Count,Output Lines Count,Desired Lines Count,All Desired Lines Extracted
0,1,1,"let s = v in let s = StartToPosRegion(s, Regex...",../tool/samples/C#/NancyFx@Nancy/failed/375800...,no extraction found for this input,Nancy.Testing.Tests.BrowserFixture.Should_retu...,0.027049,0.0,False,00.24.078,...,3256,1,1.0,PBE,0,1,29,1,29,False
1,2,2,NaN,../tool/samples/C#/NancyFx@Nancy/failed/377324...,no program found,Nancy.Testing.Tests.BrowserFixture.Should_retu...,0.012882,0.0,False,00.08.105,...,3096,1,1.0,PBE,0,1,15,1,15,False
2,3,3,NaN,../tool/samples/C#/NancyFx@Nancy/failed/382556...,no program found,Nancy.Testing.Tests.BrowserFixture.Should_retu...,0.012739,0.0,False,00.12.899,...,3126,1,1.0,PBE,0,1,15,1,15,False
3,4,4,NaN,../tool/samples/C#/NancyFx@Nancy/failed/386536...,no program found,Nancy.Testing.Tests.BrowserFixture.Should_retu...,0.012739,0.0,False,00.17.907,...,3081,1,1.0,PBE,0,1,15,1,15,False
4,5,5,NaN,../tool/samples/C#/NancyFx@Nancy/failed/405164...,no program found,Nancy.Testing.Tests.BrowserFixture.Should_retu...,0.012882,0.0,False,00.22.948,...,3087,1,1.0,PBE,0,1,15,1,15,False


In [14]:
result_big

,v,Example Count,LearnedProgram,TestInputPath,Test Output,Desired Test Output,Accuracy,IoU,Successful,Learning Duration,...,TestInputLineCount,TestCategory,ContextSizeFactor,Technique,True Positives Count,False Positives Count,False Negatives Count,Output Lines Count,Desired Lines Count,All Desired Lines Extracted
0,1,1,"let s = v in let s = StartToPosRegion(s, Regex...",../tool/samples/C#/NancyFx@Nancy/failed/375800...,no extraction found for this input,Nancy.Testing.Tests.BrowserFixture.Should_retu...,0.027049,0.000000,False,00.24.078,...,3256,1,1.0,PBE,0,1,29,1,29,False
1,2,2,NaN,../tool/samples/C#/NancyFx@Nancy/failed/377324...,no program found,Nancy.Testing.Tests.BrowserFixture.Should_retu...,0.012882,0.000000,False,00.08.105,...,3096,1,1.0,PBE,0,1,15,1,15,False
2,3,3,NaN,../tool/samples/C#/NancyFx@Nancy/failed/382556...,no program found,Nancy.Testing.Tests.BrowserFixture.Should_retu...,0.012739,0.000000,False,00.12.899,...,3126,1,1.0,PBE,0,1,15,1,15,False
3,4,4,NaN,../tool/samples/C#/NancyFx@Nancy/failed/386536...,no program found,Nancy.Testing.Tests.BrowserFixture.Should_retu...,0.012739,0.000000,False,00.17.907,...,3081,1,1.0,PBE,0,1,15,1,15,False
4,5,5,NaN,../tool/samples/C#/NancyFx@Nancy/failed/405164...,no program found,Nancy.Testing.Tests.BrowserFixture.Should_retu...,0.012882,0.000000,False,00.22.948,...,3087,1,1.0,PBE,0,1,15,1,15,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835,1,1,1,VimL/neovim@neovim/failed/567659224.log,[32m[2m[ RUN ][0m[0m put command Visual p...,ERROR: AddressSanitizer: SEGV on unknown addre...,0.025354,0.000000,False,00.00.002,...,9882,1,2.5,RLR,0,2,88,2,88,False
1836,2,2,44.5,VimL/neovim@neovim/failed/567670361.log,-- Installing: /home/travis/nvim-install/share...,ERROR: AddressSanitizer: SEGV on unknown addre...,0.153751,0.000000,False,00.00.000,...,9916,1,2.5,RLR,0,111,88,111,88,False
1837,3,3,59,VimL/neovim@neovim/failed/567681636.log,[32m[2m[ RUN ][0m[0m NULL list does not c...,ERROR: AddressSanitizer: SEGV on unknown addre...,0.198223,0.009009,False,00.00.000,...,9881,1,2.5,RLR,2,145,86,147,88,False
1838,4,4,66.25,VimL/neovim@neovim/failed/567681970.log,[119/264] Generating auto/tui/terminfo.../../i...,ERROR: AddressSanitizer: SEGV on unknown addre...,0.195000,0.004065,False,00.00.000,...,9868,1,2.5,RLR,1,163,87,164,88,False


In [7]:
result = result_big[result_big['ContextSizeFactor'] == 1]
result.head()

,v,Example Count,LearnedProgram,TestInputPath,Test Output,Desired Test Output,Accuracy,IoU,Successful,Learning Duration,...,TestInputLineCount,TestCategory,ContextSizeFactor,Technique,True Positives Count,False Positives Count,False Negatives Count,Output Lines Count,Desired Lines Count,All Desired Lines Extracted
0,1,1,"let s = v in let s = StartToPosRegion(s, Regex...",../tool/samples/C#/NancyFx@Nancy/failed/375800...,no extraction found for this input,Nancy.Testing.Tests.BrowserFixture.Should_retu...,0.027049,0.0,False,00.27.748,...,NaN,NaN,1.0,PBE,0,1,29,1,29,False
1,2,2,NaN,../tool/samples/C#/NancyFx@Nancy/failed/377324...,no program found,Nancy.Testing.Tests.BrowserFixture.Should_retu...,0.012882,0.0,False,00.09.160,...,NaN,NaN,1.0,PBE,0,1,15,1,15,False
2,3,3,NaN,../tool/samples/C#/NancyFx@Nancy/failed/382556...,no program found,Nancy.Testing.Tests.BrowserFixture.Should_retu...,0.012739,0.0,False,00.14.946,...,NaN,NaN,1.0,PBE,0,1,15,1,15,False
3,4,4,NaN,../tool/samples/C#/NancyFx@Nancy/failed/386536...,no program found,Nancy.Testing.Tests.BrowserFixture.Should_retu...,0.012739,0.0,False,00.21.461,...,NaN,NaN,1.0,PBE,0,1,15,1,15,False
4,5,5,NaN,../tool/samples/C#/NancyFx@Nancy/failed/405164...,no program found,Nancy.Testing.Tests.BrowserFixture.Should_retu...,0.012882,0.0,False,00.27.039,...,NaN,NaN,1.0,PBE,0,1,15,1,15,False


In [ ]:
def rtf(result,technique):
    return result[result['Technique'] == technique]

In [ ]:
# all techniques - extracted desired log part
ax = sns.countplot(x='All Desired Lines Extracted', hue='Technique', data=result, order=bool_order)
process_fig(ax, 'success', 'all', True, 'Technique')

In [ ]:
# separate technique success rate against number of examples
#ax = sns.catplot(x='ExampleCount', col='Technique', hue = 'AllDesiredExtracted',
#                 data=result, kind='count')
def plot_examples_success_technique(technique):
    ax = sns.countplot(x='Example Count', hue='All Desired Lines Extracted',
                  data=rtf(result,technique), hue_order=bool_order)
    process_fig(ax, 'success-examples', technique, True, 'All Desired\nLines Extracted')

for technique in techniques:
    plot_examples_success_technique(technique)

In [ ]:
# precision of extraction: how much of the extracted text is actually necessary
result['Precision'] = result['True Positives Count'] / result['Output Lines Count']

ax = sns.barplot(x='Technique', y='Precision', hue='All Desired Lines Extracted', data=result, hue_order=bool_order, errcolor='grey')
process_fig(ax, 'precision', 'all', False, 'All Desired\nLines Extracted')

In [ ]:
# precision one technique against example count
def precision_examples(technique):
    ax = sns.barplot(x='Example Count', y='Precision', hue='All Desired Lines Extracted', data=rtf(result, technique), hue_order=bool_order, errcolor='grey')
    process_fig(ax, 'precision', technique, False, 'All Desired\nLines Extracted')

for technique in techniques:
    precision_examples(technique)

In [ ]:
# category distribution
pbe_result = rtf(result, 'PBE')
plt.figure(figsize=(16,4))
order = list(set(pbe_result['Categories']))
order.sort()
order.sort(key = len)
ax = sns.countplot(x='Categories', data=pbe_result, order=order)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right", fontsize=13, horizontalalignment='center')
process_fig(ax, 'categories', 'dataset', True, '')

In [ ]:
def category_count(categories):
    return len(set(categories.split('-')))

In [ ]:
# single-category-precsion
result['Category Count'] = result.apply(lambda x : category_count(x['Categories']), axis=1)
result['Single Category'] = result['Category Count'] == 1
result_single_category = result[result['Single Category']]
ax = sns.barplot(x='Technique',
                 y='Precision',
                 hue='All Desired Lines Extracted',
                 data=result_single_category,
                 hue_order=bool_order,
                 errcolor='grey')
process_fig(ax, 'single-cateogry-precision', 'all', False, 'All Desired\nLines Extracted')

In [ ]:
# single category vs. multiple
ax = sns.barplot(x='Technique', y='Precision', hue='Single Category', data=result, hue_order=bool_order, errcolor='grey')
process_fig(ax, 'precision-category-singularity', 'all', False, 'Single Category')

In [ ]:
# category count
ax = sns.countplot(x='Example Count', hue='Category Count', data=rtf(result, 'PBE'))
process_fig(ax, 'categorycount-examplecount', 'dataset', True, 'Category Count')

In [ ]:
# category count vs example count
# ax = sns.barplot(x='ExampleCount', y='CategoryCount', data=rtf(result, 'pbe'))
# process_fig(ax, 'categorycount-examplecount', 'dataset', False, '')

In [ ]:
ax = sns.barplot(x='Example Count', y='Precision', hue='Category Count', data=result, errcolor='grey')
process_fig(ax, 'precision-categorycount-examplecount', 'all', False, 'Category Count')

In [ ]:
def example_count_category_count_precision(technique):
    ax = sns.barplot(x='Example Count', y='Precision', hue='Category Count', data=rtf(result, technique), errcolor='grey')
    process_fig(ax, 'precision-categorycount-examplecount', technique, False, 'Category Count')

for technique in techniques:
    example_count_category_count_precision(technique)

In [ ]:
# example count category comparing techniques in one picture
ax = sns.barplot(x='Technique', y='Precision', hue='Category Count',
                 data=result, errcolor='grey')
process_fig(ax, 'categorycount-precision', 'all', False, 'Category Count')

In [ ]:
# success of extractions against category counts
def example_count_success(technique):
    ax = sns.barplot(x='Category Count', y='Precision', hue='All Desired Lines Extracted',
                  data=rtf(result, technique), hue_order=bool_order, errcolor='grey')
    process_fig(ax, 'categorycount-success', technique, False, 'All Desired Lines Extracted')
    
for technique in techniques:
    example_count_success(technique)

In [ ]:
# pbe: if the extraction failed, why did it fail? (when was no program learned/regex did not match?)
failed = result[(result['All Desired Lines Extracted'] == False) & (result.Technique == 'PBE')]
# failed_pbe = failed
failed.loc[failed['Test Output'].isin(['no program found', 'no extraction found for this input']), 'to'] = failed['Test Output']
failed.loc[~failed['Test Output'].isin(['no program found', 'no extraction found for this input']), 'to'] = 'extraction found'
failed.loc[failed['to'] == 'no extraction found for this input', 'to'] = 'no extraction found\nfor this input'
pbe_fail_order = ['extraction found', 'no extraction found\nfor this input', 'no program found']

ax = sns.countplot(x="to", data=failed, order=pbe_fail_order)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
ax.set_xlabel('')
process_fig(ax, 'failure-reason', 'PBE', True, '')

In [ ]:
# pbe failure reason against category count
ax = sns.countplot(x='Category Count', hue='to', data=failed, hue_order=pbe_fail_order)
process_fig(ax, 'failure-reason-categorycount', 'PBE', True, 'Retrieval Result')

In [ ]:
# timed_result = result
# timed_result['Learning Duration'] = pd.to_datetime(result['Learning Duration'],format='%M.%S.%f') - pd.Timestamp('1900-01-01 00:00:00.000')
# timed_result['Application Duration'] = pd.to_datetime(result['Application Duration'],format='%M.%S.%f') - pd.Timestamp('1900-01-01 00:00:00.000')

In [ ]:
#timed_result.head()

In [ ]:
# # learning & application duration for all,
# ax = sns.barplot(x='Technique', y='Learning Duration', data=timed_result)
# process_fig(ax, 'learning-duration', 'all', False, '')

# ax = sns.barplot(x='Technique', y='Application Duration', data=timed_result)
# process_fig(ax, 'application-duration', 'all', False, '')

# # learning duration for pbe,
# ax = sns.barplot(x='Example Count', y='Learning Duration',
#                  hue='Category Count', data=rtf(timed_result, 'PBE'))
# process_fig(ax, 'learning-duration-examplecount', 'pbe', False, 'Category Count')

# # learning duration for pbe per type?